In [7]:
import os
import torch
import pandas as pd 
import numpy as np 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from dataloader import TrackingDistanceData

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [10]:

training_dataset = TrackingDistanceData()

features = training_dataset.x
labels = training_dataset.y

#features, labels = dataset[0]

print(features)
print(labels)


# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = features.to("cuda")

model = NeuralNetwork().to(device)
print(model)

train_dataloader = DataLoader(training_dataset, batch_size=64)
test_dataloader = DataLoader(training_dataset, batch_size=64)



loss_fn = nn.NLLLoss()
learning_rate = 1e-3
batch_size = 64
epochs = 5
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


tensor([0, 0, 0,  ..., 0, 0, 0])
tensor([[32.1519, 29.6866, 32.7681, 52.8125],
        [32.1519, 29.6866, 32.7681, 52.8125],
        [32.1519, 29.6866, 32.7681, 52.8125],
        ...,
        [24.6235, 60.1576, 42.9591, 28.2484],
        [24.6235, 60.1864, 42.9497, 28.2484],
        [24.6235, 60.2041, 42.9783, 28.2484]])
tensor([[32.1519, 29.6866, 32.7681, 52.8125],
        [32.1519, 29.6866, 32.7681, 52.8125],
        [32.1519, 29.6866, 32.7681, 52.8125],
        ...,
        [24.6235, 60.1576, 42.9591, 28.2484],
        [24.6235, 60.1864, 42.9497, 28.2484],
        [24.6235, 60.2041, 42.9783, 28.2484]])
tensor([0, 0, 0,  ..., 0, 0, 0])
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=2, bias=True)
  )
)


In [19]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            if any(pred.argmax(1) == 0):
                print("found a 1-------------------------------------------------------------------------------------------------------------------------")    
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: -37.846699  [   64/141156]
loss: -12.635714  [ 6464/141156]
loss: -37.929447  [12864/141156]
loss: -38.029396  [19264/141156]
loss: -38.129341  [25664/141156]
loss: -38.229286  [32064/141156]
loss: -38.329231  [38464/141156]
loss: -38.429176  [44864/141156]
loss: -12.814757  [51264/141156]
loss: -38.511925  [57664/141156]
loss: -38.611870  [64064/141156]
loss: -12.904902  [70464/141156]
loss: -38.694633  [76864/141156]
loss: -38.794567  [83264/141156]
loss: -38.894524  [89664/141156]
loss: -38.994453  [96064/141156]
loss: -39.094414  [102464/141156]
loss: -12.981104  [108864/141156]
loss: -13.081144  [115264/141156]
loss: -39.183125  [121664/141156]
loss: -39.283070  [128064/141156]
loss: -39.383015  [134464/141156]
loss: -39.482964  [140864/141156]
found a 0-------------------------------------------------------------------------------------------------------------------------
found a 0------------------------------------------------------

KeyboardInterrupt: 